## C03 Making Fast Function Calls
### Using Globals
> Don't use globals in performance critical parts of code

#### The trouble with globals
Compiler has been unable to infer the type of result when working with the global variable, marking it as `Any`

In [2]:
using BenchmarkTools

In [1]:
p = 2 
function pow_array(x::Vector{Float64})
    s = 0.0;
    for y in x
        s += y^p
    end
    return s
end

pow_array (generic function with 1 method)

In [3]:
t = rand(100000);
@btime pow_array(t);

  3.448 ms (300000 allocations: 4.58 MiB)


In [4]:
@code_warntype pow_array(t)

MethodInstance for pow_array(::Vector{Float64})
  from pow_array(x::Vector{Float64}) @ Main ~/Documents/code/Jupyter/Julia-HPC/03-Functions.ipynb:2
Arguments
  #self#::Core.Const(pow_array)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Any
  y::Float64
Body::Any
1 ─

(s = 0.0)
│   %

2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Any
│   %11 = (y ^ Main.p)::Any
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s



#### Fixing performance issues with globals
There are two ways to correct the issues of globals
1. `const p`: which will constrain the type of variable p.
2. `pow(array::Vector{Float64}, p)`: make the `p` being the argument of function, during which the type of `p` will be inference.

In [8]:
const p2 = 2
function pow_array2(x::Vector{Float64})
    s = 0.0
    for y in x
        s += y^p2
    end
    return s
end

pow_array2 (generic function with 1 method)

In [9]:
@btime pow_array2(t);

  249.625 μs (0 allocations: 0 bytes)


In [7]:
@code_warntype pow_array2(t)

MethodInstance for pow_array2(::Vector{Float64})
  from pow_array2(x::Vector{Float64}) @ Main ~/Documents/code/Jupyter/Julia-HPC/03-Functions.ipynb:2
Arguments
  #self#::Core.Const(pow_array2)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  y::Float64
Body::Float64
1 ─       (s = 0.0)
│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Float64
│   %11 = (y ^ Main.p2)::Float64
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s



In [10]:
function pow_array3(x::Vector{Float64})
    return pow_array_inner(x,p)
end

function pow_array_inner(x, pow)
    s = 0.0;
    for y in x
        s += y^pow
    end
    return s
end

pow_array_inner (generic function with 1 method)

In [11]:
@btime pow_array3(t);

  249.666 μs (1 allocation: 16 bytes)


#### Inlining
